In [1]:
import pygame, sys, time, random
import cv2
import numpy as np
import base64
import os.path
import pandas as pd 

pygame 2.0.0.dev6 (SDL 2.0.10, python 3.7.6)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:


class Game:


    def __init__(self):
        
        # Difficulty settings
        # Easy      ->  10
        # Medium    ->  25
        # Hard      ->  40
        # Harder    ->  60
        # Impossible->  120
        self.difficulty = 10

        # Window size
        self.frame_size_x = 150
        self.frame_size_y = 150
        # Colors (R, G, B)
        self.black = pygame.Color(0, 0, 0)
        self.white = pygame.Color(255, 255, 255)
        self.red = pygame.Color(255, 0, 0)
        self.green = pygame.Color(0, 255, 0)
        self.blue = pygame.Color(0, 0, 255)
        self.mag = pygame.Color(255, 0, 255)
        self.imgresh1 = None
        self.imgresh2 = None
        self.initialize()
        self.reward = 0
        
        self.path = r"C:\Users\Flo\Studium\Sem5\Bilder"
        self.filename = "data.csv"

        
    def initialize(self):
        # Checks for errors encountered
        self.check_errors = pygame.init()
        # pygame.init() example output -> (6, 0)
        # second number in tuple gives number of errors
        if self.check_errors[1] > 0:
            print(f'[!] Had {check_errors[1]} errors when initialising game, exiting...')
            sys.exit(-1)
        else:
            print('[+] Game successfully initialised')

         
        # Initialise game window
        pygame.display.set_caption('Snake Eater')
        self.game_window = pygame.display.set_mode((self.frame_size_x, self.frame_size_y)) 

        
                # FPS (frames per second) controller
        self.fps_controller = pygame.time.Clock()


        # Game variables
        self.snake_pos = [100, 50]
        self.snake_body = [[100, 50], [100-10, 50], [100-(2*10), 50]]

        self.food_pos = [random.randrange(1, (self.frame_size_x//10)) * 10, random.randrange(1, (self.frame_size_y//10)) * 10]
        self.food_spawn = True

        self.direction = 'RIGHT'
        self.change_to = self.direction

        self.score = 0

        self.event_happened = False


    def run(self, i):
        j = 0
        # Main logic
        while True:
            
            img1 = np.frombuffer(pygame.image.tostring(self.game_window, "RGB"), dtype=np.uint8)
            self.imgresh1 = np.reshape(img1,(self.frame_size_x,self.frame_size_y, 3))
            
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    pygame.quit()
                    return
                    sys.exit()
                # Whenever a key is pressed down
                elif event.type == pygame.KEYDOWN:
                    self.event_happened = True
                    #cv2.imwrite("screenI.jpg", self.imgresh1)
                    # W -> Up; S -> Down; A -> Left; D -> Right
                    if event.key == pygame.K_UP or event.key == ord('w'):
                        self.change_to = 'UP'
                    if event.key == pygame.K_DOWN or event.key == ord('s'):
                        self.change_to = 'DOWN'
                    if event.key == pygame.K_LEFT or event.key == ord('a'):
                        self.change_to = 'LEFT'
                    if event.key == pygame.K_RIGHT or event.key == ord('d'):
                        self.change_to = 'RIGHT'

                    # Esc -> Create event to quit the game
                    if event.key == pygame.K_ESCAPE:
                        pygame.event.post(pygame.event.Event(pygame.QUIT))

            # Making sure the snake cannot move in the opposite direction instantaneously
            if self.change_to == 'UP' and self.direction != 'DOWN':
                self.direction = 'UP'
            if self.change_to == 'DOWN' and self.direction != 'UP':
                self.direction = 'DOWN'
            if self.change_to == 'LEFT' and self.direction != 'RIGHT':
                self.direction = 'LEFT'
            if self.change_to == 'RIGHT' and self.direction != 'LEFT':
                self.direction = 'RIGHT'

            # Moving the snake
            if self.direction == 'UP':
                self.snake_pos[1] -= 10
            if self.direction == 'DOWN':
                self.snake_pos[1] += 10
            if self.direction == 'LEFT':
                self.snake_pos[0] -= 10
            if self.direction == 'RIGHT':
                self.snake_pos[0] += 10

            # Snake body growing mechanism
            self.snake_body.insert(0, list(self.snake_pos))
            if self.snake_pos[0] == self.food_pos[0] and self.snake_pos[1] == self.food_pos[1]:
                self.score += 1
                self.reward = 10
                self.food_spawn = False
            else:
                self.snake_body.pop()
                self.reward = 0

            # Spawning food on the screen
            if not self.food_spawn:
                self.food_pos = [random.randrange(1, (self.frame_size_x//10)) * 10, random.randrange(1, (self.frame_size_y//10)) * 10]
            self.food_spawn = True

            # GFX
            self.game_window.fill(self.black)
            n = 0
            for pos in self.snake_body:
                # Snake body
                # .draw.rect(play_surface, color, xy-coordinate)
                # xy-coordinate -> .Rect(x, y, size_x, size_y)
                if n == 0:
                    pygame.draw.rect(self.game_window, self.mag, pygame.Rect(pos[0], pos[1], 10, 10))
                else:
                    pygame.draw.rect(self.game_window, self.green, pygame.Rect(pos[0], pos[1], 10, 10))
                n=+1
                

            # Snake food
            pygame.draw.rect(self.game_window, self.white, pygame.Rect(self.food_pos[0], self.food_pos[1], 10, 10))

            # Game Over conditions
            # Getting out of bounds
            if self.snake_pos[0] < 0 or self.snake_pos[0] > self.frame_size_x-10:
                self.game_over(i,j)
                return
            if self.snake_pos[1] < 0 or self.snake_pos[1] > self.frame_size_y-10:
                self.game_over(i,j)
                return
            # Touching the snake body
            for block in self.snake_body[1:]:
                if self.snake_pos[0] == block[0] and self.snake_pos[1] == block[1]:
                    self.game_over(i,j)
                    return

            #self.show_score(1, self.white, 'consolas', 20)
            # Refresh game screen
            pygame.display.update()

            img2 = np.frombuffer(pygame.image.tostring(self.game_window, "RGB"), dtype=np.uint8)
            self.imgresh2 = np.reshape(img2,(self.frame_size_x,self.frame_size_y, 3))
            
            #if self.event_happened == True:
            #    self.event_happened = False
                #pygame.image.save(self.game_window, "screenshot.png")
            #    cv2.imwrite("screenII.jpg", self.imgresh2)
            # Refresh rate
            self.fps_controller.tick(self.difficulty)
            self.write(i,j,self.imgresh1, self.direction, self.reward,self.imgresh2, False)
            j += 1

            
            
    # Game Over
    def game_over(self,i,j):
        self.reward = -1
        self.game_window.fill(self.black)
        pygame.display.flip()
        img2 = np.frombuffer(pygame.image.tostring(self.game_window, "RGB"), dtype=np.uint8)
        self.imgresh2 = np.reshape(img2,(self.frame_size_x,self.frame_size_y, 3))
        self.write(i,j,self.imgresh1, self.direction, self.reward,self.imgresh2, True)
        
        time.sleep(1)
        pygame.quit()



    # Score
    def show_score(self, choice, color, font, size):
        score_font = pygame.font.SysFont(font, size)
        score_surface = score_font.render('Score : ' + str(self.score), True, color)
        score_rect = score_surface.get_rect()
        if choice == 1:
            score_rect.midtop = (self.frame_size_x/10, 15)
        else:
            score_rect.midtop = (self.frame_size_x/2, self.frame_size_y/1.25)
        self.game_window.blit(score_surface, score_rect)
        
    # storing the data into a csv file 
    def write(self, i, j, currentstate, action, reward, nextstate, terminated): 

        
        currentre = cv2.resize(currentstate, (50,50)) 
        nextre = cv2.resize(nextstate, (50, 50)) 
        
        cv2.imwrite(os.path.join(self.path,"current_{}_{}.png".format(i,j)), currentre)
        cv2.imwrite(os.path.join(self.path,"next_{}_{}.png".format(i,j)), nextre)
        

        data = [{'currentstate': "current_{}_{}.png".format(i,j), 'action': action, 'reward': reward, 'nextstate':"next_{}_{}.png".format(i,j), 'terminated':terminated}] 
        
        if os.path.isfile(os.path.join(self.path, self.filename)): 

            df = pd.read_csv(os.path.join(self.path, self.filename), index_col = 0) 

            latest = pd.DataFrame(data) 

            df = pd.concat((df, latest), ignore_index = True, sort = False) 

        else: 

            df = pd.DataFrame(data) 

        df.to_csv(os.path.join(self.path, self.filename)) 



In [3]:
#game = Game()
#game.run(0)

In [4]:
game = Game()
for i in range(4):
    game.initialize()
    game.run(i)

[+] Game successfully initialised
[+] Game successfully initialised
[+] Game successfully initialised
[+] Game successfully initialised
[+] Game successfully initialised


In [5]:
pygame.quit()

In [6]:
print(data)

NameError: name 'data' is not defined